In [9]:
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
import warnings
warnings.filterwarnings("ignore")

In [10]:
import os.path
def path_base(base_name):
    current_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    print(current_dir)
    data_dir = current_dir.replace('notebook','data')
    print(data_dir)
    data_base = data_dir + '\\' + base_name
    print(data_base)
    return data_base

In [11]:
base = pd.read_csv(path_base('db_dados_credito.csv'))

C:\MyPhyton\DataScience\notebook
C:\MyPhyton\DataScience\data
C:\MyPhyton\DataScience\data\db_dados_credito.csv


In [12]:
base.head(3)

,clientid,income,age,loan,default
0,1,66155.92510,59.01702,8106.53213,0
1,2,34415.15397,48.11715,6564.74502,0
2,3,57317.17006,63.10805,8020.95330,0


In [5]:
base.describe()

,clientid,income,age,loan,default
count,2000.00000,2000.00000,1997.00000,2000.00000,2000.00000
mean,1000.50000,45331.60002,40.80756,4444.36969,0.14150
std,577.49459,14326.32712,13.62447,3045.41002,0.34862
min,1.00000,20014.48947,-52.42328,1.37763,0.00000
25%,500.75000,32796.45972,28.99042,1939.70885,0.00000
50%,1000.50000,45789.11731,41.31716,3974.71942,0.00000
75%,1500.25000,57791.28167,52.58704,6432.41062,0.00000
max,2000.00000,69995.68558,63.97180,13766.05124,1.00000


In [33]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
clientid    2000 non-null int64
income      2000 non-null float64
age         1997 non-null float64
loan        2000 non-null float64
default     2000 non-null int64
dtypes: float64(3), int64(2)
memory usage: 78.2 KB


# Tratamento de dados inconsistentes

In [34]:
base.loc[base['age']<=0]

,clientid,income,age,loan,default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


In [35]:
print(base.loc[15].age)
print(base.loc[21].age)
print(base.loc[26].age)

-28.218361321371003
-52.423279919661596
-36.4969755136408


In [36]:
# 1-Apagar coluna
# base.drop('age',1,inplace=True)
# Ou 2-Apagar registros
# base.drop(base[base.age <=0].index,inplace=True)
# Ou 3-Preencher manualmente
# Ou 4-Preencher com a média
base.mean()
base.age[base.age>0].mean()

40.92770044906149

In [37]:
base.loc[base['age']<=0,'age'] = base.age[base.age>0].mean()

In [38]:
print(base.loc[0].age)
print(base.loc[15].age)
print(base.loc[21].age)
print(base.loc[26].age)

59.017015066929204
40.92770044906149
40.92770044906149
40.92770044906149


# Valores nulos

In [39]:
base.loc[pd.isnull(base.age)]

,clientid,income,age,loan,default
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


In [40]:
## Previsores e classe

In [41]:
previsores = base.iloc[:,1:4]
previsores.head(3)

,income,age,loan
0,66155.925095,59.017015,8106.532131
1,34415.153966,48.117153,6564.745018
2,57317.170063,63.108049,8020.953296


In [42]:
classe = base.iloc[:,4]
classe.head(3)

0    0
1    0
2    0
Name: default, dtype: int64

In [43]:
import numpy as np
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')


In [44]:
imputer = imputer.fit(previsores.iloc[:, 0:3])
previsores.iloc[:, 0:3] = imputer.transform(previsores.iloc[:,0:3])

In [45]:
print(previsores.loc[0].age)
print(previsores.loc[28].age)
print(previsores.loc[30].age)
print(previsores.loc[31].age)
previsores.head(3)

59.017015066929204
40.92770044906149
40.92770044906149
40.92770044906149


,income,age,loan
0,66155.925095,59.017015,8106.532131
1,34415.153966,48.117153,6564.745018
2,57317.170063,63.108049,8020.953296


# Escalonamento de variáveis
Standardisation x = (x - média) / desvio

In [46]:
print(previsores.loc[28].age)
print(previsores.loc[30].age)
print(previsores.loc[31].age)

40.92770044906149
40.92770044906149
40.92770044906149


In [47]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [48]:
print(scaler.fit(previsores))
print(scaler.mean_)
print(scaler.fit_transform(previsores))
previsores = scaler.fit_transform(previsores)

StandardScaler(copy=True, with_mean=True, with_std=True)
[4.53316000e+04 4.09277004e+01 4.44436969e+03]
[[ 1.45393393  1.36538005  1.20281942]
 [-0.76217555  0.54265932  0.69642695]
 [ 0.83682073  1.67417101  1.17471147]
 ...
 [-0.07122592 -0.97448606  0.35420081]
 [-0.11000289  1.73936652 -0.92675625]
 [ 1.682986    1.14917551  0.96381038]]


In [49]:
type(previsores)

numpy.ndarray

# Divisão em base de treinos e testes

In [50]:
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.15, random_state=0)

In [51]:
print(previsores_treinamento.shape)
print(classe_treinamento.shape)

(1700, 3)
(1700,)


In [52]:
print(previsores_teste.shape)
print(classe_teste.shape)

(300, 3)
(300,)


In [63]:
#importação da biblioteca
#criacao do classificador
from sklearn.tree import DecisionTreeClassifier
classificador = DecisionTreeClassifier(criterion='entropy',random_state=0)

In [64]:
classificador.fit(previsores_treinamento,classe_treinamento)
previsoes = classificador.predict(previsores_teste)

In [65]:
from sklearn.metrics import confusion_matrix,accuracy_score
precisao = accuracy_score(classe_teste,previsoes)
matriz = confusion_matrix(classe_teste,previsoes)

In [66]:
precisao

0.9733333333333334

In [67]:
matriz

array([[254,   6],
       [  2,  38]], dtype=int64)